## 다음 문제를 풀이하시오.

diabetes.csv 파일을 대상으로 다음 문제를 풀이하시오.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("diabetes.csv")
df.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0

### Q1. 당뇨 여부인 Outcome과 임신 횟수인 Pregnancies을 제외한 수치형 변수는  결측치가 0으로 기록이 되어 있다. 모든 변수의 결측치 개수를 더한 개수를 기술하시오.
(정답 예시: 123)

In [3]:
# df_na = df.drop(["Outcome", "Pregnancies"], axis = 1)
df_na = df.iloc[:, 1:-1]
df_na.head(1)

Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0      148             72             35        0  33.6   

   DiabetesPedigreeFunction  Age  
0                     0.627   50

In [4]:
ser = pd.Series([0, 2, 3, 0, 2])

In [5]:
np.where(ser == 0, np.nan, ser)

array([nan,  2.,  3., nan,  2.])

In [6]:
ser.replace(to_replace = 0, value = np.nan)

0    NaN
1    2.0
2    3.0
3    NaN
4    2.0
dtype: float64

In [7]:
sum(df_na["Glucose"] == 0)

5

In [25]:
x = df_na["Glucose"]
sum(x == 0)

5

In [8]:
df_na.apply(lambda x: sum(x == 0))

Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
dtype: int64

In [9]:
df_na.apply(lambda x: sum(x == 0)).sum()

652

### Q2. Outcome 변수를 제외한 변수간 상관분석을 실시하고 상관계수가 가장 높은 조합의 변수명과 상관계수를 기술하시오.
 - 변수명은 알파벳 순서대로 기술한다.
 - 상관계수는 반올림하여 소수점 셋째자리까지 기술한다.
 
(정답 예시: aaa, bbb, 0.123)

In [10]:
df_q2 = df.drop("Outcome", axis = 1).copy()
df_q2.head(1)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   

   DiabetesPedigreeFunction  Age  
0                     0.627   50

In [27]:
df_q2.corr().round(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  \
Pregnancies                      1.00     0.13           0.14          -0.08   
Glucose                          0.13     1.00           0.15           0.06   
BloodPressure                    0.14     0.15           1.00           0.21   
SkinThickness                   -0.08     0.06           0.21           1.00   
Insulin                         -0.07     0.33           0.09           0.44   
BMI                              0.02     0.22           0.28           0.39   
DiabetesPedigreeFunction        -0.03     0.14           0.04           0.18   
Age                              0.54     0.26           0.24          -0.11   

                          Insulin   BMI  DiabetesPedigreeFunction   Age  
Pregnancies                 -0.07  0.02                     -0.03  0.54  
Glucose                      0.33  0.22                      0.14  0.26  
BloodPressure                0.09  0.28                      0.04  0.24  
SkinThickness                0.44  0.39                      0.18 -0.11  
Insulin                      1.00  0.20                      0.19 -0.04  
BMI                          0.20  1.00                      0.14  0.04  
DiabetesPedigreeFunction     0.19  0.14                      1.00  0.03  
Age                         -0.04  0.04                      0.03  1.00

In [12]:
df_q2_corr = df_q2.corr()
type(df_q2_corr)

pandas.core.frame.DataFrame

In [13]:
df_q2_corr.apply(lambda x: x[x != 1].max())

Pregnancies                 0.544341
Glucose                     0.331357
BloodPressure               0.281805
SkinThickness               0.436783
Insulin                     0.436783
BMI                         0.392573
DiabetesPedigreeFunction    0.185071
Age                         0.544341
dtype: float64

In [14]:
df_q2_corr.apply(lambda x: x[x != 1].idxmax())

Pregnancies                           Age
Glucose                           Insulin
BloodPressure                         BMI
SkinThickness                     Insulin
Insulin                     SkinThickness
BMI                         SkinThickness
DiabetesPedigreeFunction          Insulin
Age                           Pregnancies
dtype: object

In [15]:
df_q2_corr_max = pd.DataFrame({"col":  df_q2_corr.apply(lambda x: x[x != 1].idxmax()),
                               "corr": df_q2_corr.apply(lambda x: x[x != 1].max())})
df_q2_corr_max = df_q2_corr_max.reset_index()
df_q2_corr_max

index            col      corr
0               Pregnancies            Age  0.544341
1                   Glucose        Insulin  0.331357
2             BloodPressure            BMI  0.281805
3             SkinThickness        Insulin  0.436783
4                   Insulin  SkinThickness  0.436783
5                       BMI  SkinThickness  0.392573
6  DiabetesPedigreeFunction        Insulin  0.185071
7                       Age    Pregnancies  0.544341

In [16]:
df_q2_corr_max.loc[df_q2_corr_max["corr"] == df_q2_corr_max["corr"].max(), ]

index          col      corr
0  Pregnancies          Age  0.544341
7          Age  Pregnancies  0.544341

### Q3. 임신 경험 여부와 BMI간의 관계를 보고자 한다. 이때 적절한 검정방법을 사용하고 해당 검정의 검정통계량의 절대값을 반올림하여 소수점 셋째 자리 까지 표기하시오.
(정답 예시: 1.234)

In [17]:
df_q3 = df.loc[:, ["Pregnancies", "BMI"]].copy()
df_q3.head(2)

Pregnancies   BMI
0            6  33.6
1            1  26.6

In [18]:
from scipy.stats import ttest_ind

In [19]:
stat, p = ttest_ind(df_q3.loc[df_q3["Pregnancies"] == 0, "BMI"],
                    df_q3.loc[df_q3["Pregnancies"] >  0, "BMI"])
round(abs(stat), 3)

3.341

### Q4. 임신 횟수, 혈당, 혈압, 피부 두께로 BMI를 설명할 수 있는지 확인해보려 한다. BMI를 종속변수로 한 다중선형 회귀분석을 실시하고 유의미한 변수의 개수와 결정계수를 기술하시오.
 - 결측치는 모두 제거하고 분석을 실시한다.
 - 결정계수는 소수점 둘 째 자리까지 기술한다.

(정답 예시: 1, 0.12)

In [20]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [40]:
df_q4 = df.loc[:, ["Pregnancies", "Glucose", "BloodPressure",
                   "SkinThickness", "BMI"]].copy()
df_q4.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0            6      148             72             35  33.6
1            1       85             66             29  26.6

In [41]:
len(df_q4)

768

In [42]:
for col in df_q4.columns[1:]:
    df_q4 = df_q4.loc[df_q4[col] != 0, :]
len(df_q4)

532

In [43]:
df_q4.apply(lambda x: sum(x == 0))

Pregnancies      77
Glucose           0
BloodPressure     0
SkinThickness     0
BMI               0
dtype: int64

In [32]:
df_q4 = df.loc[:, ["Pregnancies", "Glucose", "BloodPressure",
                   "SkinThickness", "BMI"]].copy()
df_q4.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0            6      148             72             35  33.6
1            1       85             66             29  26.6

In [ ]:
# df_q4["Pregnancies"].replace() # to_replace, value
df_q4.apply("replace", to_replace = 0, value = np.nan).head()

In [ ]:
df_q4 = pd.concat([df_q4["Pregnancies"], 
                   df_q4.iloc[:, 1:].apply("replace", to_replace = 0, value = np.nan)],
                  axis = 1)
df_q4.head()

In [37]:
df_q4.apply(lambda x: x.isna().sum())

Pregnancies        0
Glucose            5
BloodPressure     35
SkinThickness    227
BMI               11
dtype: int64

In [38]:
df_q4 = df_q4.dropna()

In [39]:
len(df_q4)

532

In [44]:
from statsmodels.formula.api import ols

In [45]:
formula = "BMI ~ " + " + ".join(df_q4.columns[:-1])
formula

'BMI ~ Pregnancies + Glucose + BloodPressure + SkinThickness'

In [46]:
model = ols(formula, data = df_q4).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BMI   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     112.5
Date:                Fri, 28 Aug 2020   Prob (F-statistic):           2.86e-69
Time:                        09:28:41   Log-Likelihood:                -1616.3
No. Observations:                 532   AIC:                             3243.
Df Residuals:                     527   BIC:                             3264.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        12.9673      1.447      8.962      0.000      10.125      15.810
Pregnancies      -0.1960      0.068     -2.875      0.004      -0.330      -0.062
Glucose           0.0190      0.007      2.556      0.011       0.004       0.034
BloodPressure     0.0966      0.019      5.097      0.000       0.059       0.134
SkinThickness     0.3910      0.022     17.875      0.000       0.348       0.434
==============================================================================
Omnibus:                       42.530   Durbin-Watson:                   1.901
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              138.189
Skew:                           0.304   Prob(JB):                     9.83e-31
Kurtosis:                       5.422   Cond. No.                         964.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## ~ 9:45

### Q5. 결측치 처리에 따른 효과를 알아보고자 한다. 4번에서 사용된 독립변수 중 결측치를 모두 해당 변수의 평균치로 치환하고 다중선형 회귀분석을 실시하여 얻은 결정계수와 결측치를 연령대 평균 값으로 치환한 후 다중선형 회귀분석을 실시하여 얻은 결정계수를 차례대로 기술하시오.
 - 단, 이 분석은 70세 미만의 데이터만 사용한다.
 - 연령대 구분 단위는 10살 단위로 한다(17 => 1, 30 => 3, 45 => 4)
 - 결정계수는 소수점 둘 째 자리까지 기술한다.

(정답 예시: 0.12, 0.34)

### Q6. 분류모델의 성능을 비교하고자 한다. Outcome 변수를 독립변수로 하고 나머지 변수를 독립변수로 하고, 독립변수의 경우 결측치로 간주되는 것은 모두 제거한다. 이항 로지스틱 회귀분석과, 의사결정나무로 학습을 하고 그 정확도를 각각 기술하시오.
 - 학습은 최초 100개 데이터를 제외한 나머지로 실시한다.
 - 평가는 최초 100개 데이터로 실시한다.
 - 이항 로지스틱 회귀분석의 경우 유의미 하지 않은 변수는 제거한다.
 - 이항 로지스틱 회귀분석의 문턱값은 0.5로 설정한다.
 - 정확도는 반올림하여 소수점 셋째 자리까지 표기한다.
 
(정답 예시: 0.123, 0.456)